# tourapi

In [ ]:
## culturalfacilities 

In [ ]:
import pandas as pd
tourapi = pd.read_excel('data/dataset/tourAPI/culturalFacilities.xls')

In [ ]:
tourapi['source'] = 'ToruAPI'

In [3]:
tourapi['category'] = "cultural_facilities"

In [4]:
tourapi.head(2)

,명칭,우편번호,관리자,전화번호,주소,위도,경도,개요,문의 및 안내,규모,...,이용요금,할인정보,관람소요시간,주차시설,주차요금,유모차 대여 여부,애완동물 동반 가능 여부,신용카드 가능 여부,상세정보,category
0,금나래아트홀,8611,NaN,NaN,서울특별시 금천구 시흥대로73길 70 금천구청종합청사,37.456493,126.895521,"금나래아트홀은 클래식, 연극, 뮤지컬 등 다양한 장르의 공연을 소화할 수 있는 다목...",NaN,"지상 2층, 지하 2층",...,공연마다 상이함,NaN,NaN,가능 (금천구청 지하주차장),1시간 무료 (1시간 이후 10분당 500원),없음,NaN,가능,"대관안내:공연장, 갤러리 대관 가능<br />\n※ 대관문의 : 금나래 아트홀 운영...",cultural_facilities
1,상상톡톡 미술관,1228,NaN,NaN,서울특별시 강북구 월계로 173(번동),37.624210,127.038297,상상 톡톡 미술관은 강북구 번동 꿈의 숲 아트센터 내에 있는 미술관을 새로 단장하여...,NaN,전시장 면적: 453 ㎡ (약 137평),...,"[입장료]<br>-12,000원<br>[3인 가족권]<br>-22,000원<br>※...",NaN,약 60분,가능 (북서울꿈의숲 서문 주차장),공연 이용 시 1시간 20분 무료 주차,없음,NaN,가능,체험프로그램:전시에 따라 체험프로그램 변동\n,cultural_facilities


In [ ]:
tourapi.rename(columns= {'명칭': 'place_name', 
                                   '주소':'address', 
                                   '위도': 'latitude', 
                                   '경도': 'longitude', 
                                   '이용시간': 'available_time', 
                                   '주차시설': 'parking', 
                                   '개요': 'description', 
                                   '우편번호': 'zip_code', ## 
                                   '관리자': 'manager',
                                   '전화번호': 'phone',
                                   '문의 및 안내': 'guidance',
                                   '규모': 'scale',
                                   '수용인원': 'capacity',
                                   '쉬는날': 'day_off',
                                   '이용요금': 'fee',
                                   '할인정보': 'discount_info',
                                   '관람소요시간': 'viewing_time',
                                   '주차요금': 'parking_fee',
                                   '유모차 대여 여부': 'stroller',
                                   '애완동물 동반 가능 여부': 'pet',
                                   '신용카드 가능 여부': 'credit_card',
                                   '상세정보': 'details'}, inplace=True)

tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'parking', 'description', 'zip_code', 'manager', 'phone', 'guidance', 'scale', 'capacity', 'day_off', 
                            'fee', 'discount_info', 'viewing_time', 'parking_fee', 'stroller', 'pet', 'credit_card', 'details']]

In [6]:
tourapi.isnull().sum() / len(tourapi)

place_name        0.000000
address           0.000740
latitude          0.000000
longitude         0.000000
available_time    0.034763
parking           0.091346
description       0.002959
zip_code          0.005178
manager           0.991494
phone             0.982249
guidance          0.991864
scale             0.649408
capacity          0.717456
day_off           0.087278
fee               0.181953
discount_info     0.874260
viewing_time      0.721524
parking_fee       0.582101
stroller          0.213018
pet               0.477811
credit_card       0.184541
details           0.254808
dtype: float64

In [7]:
tourapi.isnull().sum()

place_name           0
address              2
latitude             0
longitude            0
available_time      94
parking            247
description          8
zip_code            14
manager           2681
phone             2656
guidance          2682
scale             1756
capacity          1940
day_off            236
fee                492
discount_info     2364
viewing_time      1951
parking_fee       1574
stroller           576
pet               1292
credit_card        499
details            689
dtype: int64

In [ ]:
import openai
import pandas as pd
import re
from tqdm import tqdm  # tqdm import

import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def preprocess_day_off(day_off):

    if not day_off or pd.isna(day_off):
        return [], []

    days = []
    special_notes = []

    # 휴무일 정보 처리
    if "월요일" in day_off:
        days.append("월")
    if "화요일" in day_off:
        days.append("화")
    if "수요일" in day_off:
        days.append("수")
    if "목요일" in day_off:
        days.append("목")
    if "금요일" in day_off:
        days.append("금")
    if "토요일" in day_off:
        days.append("토")
    if "일요일" in day_off:
        days.append("일")
    if "주말" in day_off:
        days.extend(["토", "일"])
    if "공휴일" in day_off:
        days.append("공휴일")
    if "마지막 주 일요일" in day_off:
        days.append("일")
        special_notes.append("마지막 주 일요일 휴무")

    if "전화문의" in day_off:
        special_notes.append("휴무일은 전화문의 필요")

    return list(set(days)), list(set(special_notes)) 

def preprocess_available_time(available_time):

    if not available_time or pd.isna(available_time):
        return "", [] 

    time_only = re.sub(r"\s*\(.*?\)", "", available_time).strip()
    special_notes = re.findall(r"\(.*?\)", available_time)

    special_notes_cleaned = [note.strip("()").strip() for note in special_notes]
    return time_only, special_notes_cleaned

def generate_template_with_gpt(preprocessed_day_off, time_and_notes):
    available_time, special_notes = time_and_notes
    day_off_days, day_off_notes = preprocessed_day_off

    all_special_notes = special_notes + day_off_notes
    special_notes_str = ", ".join(list(set(all_special_notes))) if all_special_notes else "정보 없음"

    prompt = f"""
    다음은 문화시설의 운영 정보를 나타내는 데이터입니다:
    - 휴무일: {", ".join(day_off_days) or "정보 없음"}
    - 영업시간: {available_time or "정보 없음"}
    - 특이사항: {special_notes_str}
    
    위 정보를 기반으로 아래의 템플릿을 채워주세요:
    - 월: 영업시간
    - 화: 영업시간
    - 수: 영업시간
    - 목: 영업시간
    - 금: 영업시간
    - 토: 영업시간
    - 일: 영업시간
    - 휴무: []
    - 특이사항: []

    주의사항:
    1. 휴무일로 지정된 요일은 "휴무"로 표시하고, 영업시간을 작성하지 마세요.
    2. 영업시간에 특이한 규칙이 있으면 "특이사항"에 추가해주세요.
    3. JSON 형식으로 반환해주세요.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    reply = response['choices'][0]['message']['content']
    return reply

def process_and_add_column(df):
    results = []  

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows", unit="row"):
        day_off = preprocess_day_off(row.get("day_off"))
        available_time = row.get("available_time")
        time_and_notes = preprocess_available_time(available_time)

        try:
            template = generate_template_with_gpt(day_off, time_and_notes)
            results.append(template)
        except Exception as e:
            results.append({"error": str(e)})

    df["available_time_v1"] = results
    return df

tourapi = process_and_add_column(tourapi)

output_csv_path = "tourapi_culturalfacilities_processed.csv"
tourapi.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

print(f"데이터가 다음 경로에 저장되었습니다: {output_csv_path}")

Processing rows:   0%|          | 1/2704 [00:02<1:51:43,  2.48s/row]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
tourapi = pd.read_csv("src/collecting_data/data_analysis/tourapi_culturalfacilities_processed.csv")

In [35]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,place_name,address,latitude,longitude,available_time,parking,description,zip_code,manager,phone,...,day_off,fee,discount_info,viewing_time,parking_fee,stroller,pet,credit_card,details,available_time_v1
1545,다누리센터4D체험관,NaN,0.0,0.0,10:30/11:30/13:30/14:30/15:30/16:30(1일 총 6회 상영),가능<br>요금(무료),다누리센터4D체험관은 단양 시외버스터미널 옆에 있는 다누리센터 내 지하 1층에 있다...,NaN,NaN,NaN,...,월요일,"일반(4D 체험관) : 3,000원<br> 패키지(4D 체험관+아쿠아리움) : 12...",NaN,30분,NaN,NaN,NaN,없음,체험프로그램:해피패밀리 라이드 : 6분<br>\n서브탄광열차 : 4분\n,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:30, 1..."
2693,High Street Italia (하이 스트리트 이탈리아),서울특별시 강남구 가로수길 69 엘큐브elcube,0.0,0.0,11:00~20:00,불가,High Street Italia (하이 스트리트 이탈리아)는 이탈리아 기업 제품 ...,6028,NaN,NaN,...,"매주 월요일, 화요일",무료,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""휴무"",\n ..."


In [ ]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils 
importlib.reload(analysis_utils)

from analysis_utils import get_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# query = "서울특별시 강남구 가로수길 69 엘큐브elcube"

# geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)
# geocode

query = tourapi.iloc[2693]['address']
geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)

tourapi.loc[2693, 'latitude'] = np.float64(geocode['latitude'])
tourapi.loc[2693, 'longitude'] = np.float64(geocode['longitude'])

In [37]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,place_name,address,latitude,longitude,available_time,parking,description,zip_code,manager,phone,...,day_off,fee,discount_info,viewing_time,parking_fee,stroller,pet,credit_card,details,available_time_v1
1545,다누리센터4D체험관,NaN,0.0,0.0,10:30/11:30/13:30/14:30/15:30/16:30(1일 총 6회 상영),가능<br>요금(무료),다누리센터4D체험관은 단양 시외버스터미널 옆에 있는 다누리센터 내 지하 1층에 있다...,NaN,NaN,NaN,...,월요일,"일반(4D 체험관) : 3,000원<br> 패키지(4D 체험관+아쿠아리움) : 12...",NaN,30분,NaN,NaN,NaN,없음,체험프로그램:해피패밀리 라이드 : 6분<br>\n서브탄광열차 : 4분\n,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:30, 1..."


In [38]:
tourapi[tourapi['address'].isnull()]

,place_name,address,latitude,longitude,available_time,parking,description,zip_code,manager,phone,...,day_off,fee,discount_info,viewing_time,parking_fee,stroller,pet,credit_card,details,available_time_v1
569,잇다스페이스1 갤러리,NaN,37.472404,126.634438,11:00 - 18:00,없음,1920년대 소금 창고로 첫 숨을 텄다. 일제강점기 때 일본은 화약을 제조하는 원료...,NaN,NaN,010-5786-0777,...,매주 월요일,무료,NaN,30분,NaN,NaN,NaN,가능,대관안내:가능\n,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""11:00 - ..."
1545,다누리센터4D체험관,NaN,0.000000,0.000000,10:30/11:30/13:30/14:30/15:30/16:30(1일 총 6회 상영),가능<br>요금(무료),다누리센터4D체험관은 단양 시외버스터미널 옆에 있는 다누리센터 내 지하 1층에 있다...,NaN,NaN,NaN,...,월요일,"일반(4D 체험관) : 3,000원<br> 패키지(4D 체험관+아쿠아리움) : 12...",NaN,30분,NaN,NaN,NaN,없음,체험프로그램:해피패밀리 라이드 : 6분<br>\n서브탄광열차 : 4분\n,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:30, 1..."


In [ ]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_reverse_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# latitude = 37.472404
# longitude = 126.634438

# reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)
# reverse_geocode

latitude, longitude = tourapi.loc[569, ['latitude', 'longitude']]
reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)

tourapi.loc[569, 'address'] = reverse_geocode['address'].strip()

In [40]:
tourapi[tourapi['address'].isnull()]

,place_name,address,latitude,longitude,available_time,parking,description,zip_code,manager,phone,...,day_off,fee,discount_info,viewing_time,parking_fee,stroller,pet,credit_card,details,available_time_v1
1545,다누리센터4D체험관,NaN,0.0,0.0,10:30/11:30/13:30/14:30/15:30/16:30(1일 총 6회 상영),가능<br>요금(무료),다누리센터4D체험관은 단양 시외버스터미널 옆에 있는 다누리센터 내 지하 1층에 있다...,NaN,NaN,NaN,...,월요일,"일반(4D 체험관) : 3,000원<br> 패키지(4D 체험관+아쿠아리움) : 12...",NaN,30분,NaN,NaN,NaN,없음,체험프로그램:해피패밀리 라이드 : 6분<br>\n서브탄광열차 : 4분\n,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:30, 1..."


In [41]:
tourapi = tourapi.dropna(subset=['address'])

In [42]:
tourapi[tourapi['description'].isnull()]

,place_name,address,latitude,longitude,available_time,parking,description,zip_code,manager,phone,...,day_off,fee,discount_info,viewing_time,parking_fee,stroller,pet,credit_card,details,available_time_v1
412,잠월미술관,전라남도 함평군 해보면 산내길 60,35.172521,126.552887,10:00~17:00,있음,NaN,57116,NaN,NaN,...,매주 화요일 / 1월 1일 / 설날 / 추석,무료(체험 프로그램은 별도 비용),NaN,NaN,무료,NaN,NaN,NaN,체험프로그램:뽕나무밭 / 누에농장 / 염색 / 도예\n,"```json\n{\n ""월"": ""10:00~17:00"",\n ""화"": ..."
1142,청주랜드 어린이회관,충청북도 청주시 상당구 명암로 171 (명암동),36.649961,127.515815,[관람시간] 09:00~18:00<br>\n[입장시간] 09:00~17:00,있음(소형 170대 / 버스 18대),NaN,28311,NaN,NaN,...,매주 월요일 / 1월 1일 / 설날 / 추석,[관람시간] 09:00~18:00 <br>[입장시간] 09:00~17:00,NaN,NaN,무료,NaN,NaN,NaN,NaN,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""09:00~18..."
1411,동구인문학당,광주광역시 동구 동계천로 168-5,35.148828,126.927534,10:00~18:00,없음,NaN,61436,NaN,NaN,...,"매주 월요일, 공휴일",무료,NaN,NaN,NaN,NaN,NaN,NaN,체험프로그램:[정기프로그램]\n<br>- 영화인문학 극장\n<br>- 다락방 음악여...,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:00~18..."
1412,부산예술회관,부산광역시 남구 용소로 78 부산예술회관,35.130790,129.102516,09:00~22:00(휴게시간 12:00~13:00),있음(지하 주차장 63대 / 지상 5대),NaN,48501,NaN,NaN,...,연중무휴,공연 및 전시별 상이,NaN,NaN,무료,NaN,NaN,NaN,체험프로그램:이팔청춘 실버댄스&너도나도 방송댄스<br>\n가곡교실<br>\n김정현의...,"```json\n{\n ""월"": ""09:00~22:00"",\n ""화"": ..."
1413,송파구립 예송미술관,서울특별시 송파구 백제고분로 242 (삼전동),37.502700,127.093198,10:00~18:00,NaN,NaN,5584,NaN,NaN,...,"토요일, 일요일, 공휴일",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"```json\n{\n ""월"": ""10:00~18:00"",\n ""화"": ..."
1814,결성농요농사박물관,충청남도 홍성군 결성면 구성남로 91,36.532605,126.548960,- 하절기 09:00~18:00<br>\n- 동절기 09:00~17:00,가능,NaN,32212,NaN,NaN,...,"매주 월요일 / 화요일 / 1월 1일 / 설날·추석<br>※ 단, 월요일이 공휴일인...",무료,NaN,약 30분,무료,없음,NaN,없음,"소장품현황:농사유물 300점 / 선사유물(석기, 토기) 300점 / 백제토기 및 고...","```json\n{\n ""월"": ""휴무"",\n ""화"": ""휴무"",\n ..."
2227,김포독립운동기념관,경기도 김포시 양촌읍 양곡2로30번길 46,37.653327,126.629556,09:00~18:00,가능<br>요금(무료),NaN,10062,NaN,NaN,...,매주 월요일 / 1월 1일/ 설‧추석 당일,무료,NaN,약 30분,NaN,없음,불가,없음,체험프로그램:[교육]<br>\n- 역사강좌(10월) : 전시 연계 역사 강좌<br>...,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""09:00~18..."
2558,여수자동차극장,전라남도 여수시 소호12길 67,34.729634,127.642874,"1일 2회~3회<br>\n※ 방문 당일, 홈페이지 상영시간표 확인 필수",NaN,NaN,59670,NaN,NaN,...,연중무휴,"［평일］<br> - 월요일~목요일 : - 22,000원 / 차량1대당<br>［주말］...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"```json\n{\n ""월"": ""1일 2회~3회"",\n ""화"": ""1일..."


In [43]:
tourapi[tourapi['description'].isnull()]['place_name'].tolist()

['잠월미술관',
 '청주랜드 어린이회관',
 '동구인문학당',
 '부산예술회관',
 '송파구립 예송미술관',
 '결성농요농사박물관',
 '김포독립운동기념관',
 '여수자동차극장']

In [44]:
tourapi.loc[tourapi['place_name']=='잠월미술관', 'description'] = "잠월미술관은 전라남도 함평군 해보면 산내리에 위치한 작은 미술관으로, 부부 미술교사인 김광옥과 임혜숙 교사가 퇴직금을 담보로 지었다."
tourapi.loc[tourapi['place_name']=='청주랜드 어린이회관', 'description'] = "청주랜드 어린이회관은 충청북도 청주시에 위치한 가족과 어린이를 위한 복합문화 체험시설이다. 청주랜드 어린이회관은 충청북도 청주시에 위치한 가족과 어린이를 위한 복합문화 체험시설이다. 청주랜드 어린이회관 내 제2관은 청주랜드 어린이체험관으로 변신하였다. 청주랜드 어린이회관은 아동친화 도시를 향한 새로운 패러다임의 가족문화 체험장소이다. 청주랜드 어린이회관 옆 주차장에 주차를 한 후 청주랜드로 입구로 가면 폐품을 이용해 만든 정크아트작품을 만날 수 있다. 청주랜드 어린이회관은 청주동물원, 기후변화체험교육관, 놀이동산 등과 가까운 위치에 있다. "
tourapi.loc[tourapi['place_name']=='동구인문학당', 'description'] = "동구 인문학당은 광주 동구 동명동에 위치한 복합문화공간으로, 지역민들의 인문 배움터이자 문화 향유공간으로 큰 사랑을 받고 있다. 본채, 인문관, 공유 부엌 등 3개 건물과 연못이 있는 마당으로 이뤄져 있다. 본채, 인문관, 공유 부엌 등 3개 건물과 연못이 있는 마당으로 이뤄져 있다. 인문도시를 지향하는 동구가 다양한 인문 프로그램을 진행하기 위해 조성한 공간이다."
tourapi.loc[tourapi['place_name']=='부산예술회관', 'description'] = "부산예술회관은 지하1층 지상4층으로 연면적 4.752m² 규모로써 객석 240석의 소공연장, 전시실2개, 연습실3개소, 부산예총12개 단체의 사무실, 야외공연장, 그리고 주차장, 편의시설 등을 갖추고 있다."
tourapi.loc[tourapi['place_name']=='송파구립 예송미술관', 'description'] = "예송미술관은 송파구민의 시각예술향유를 위한 전시관으로 다양한 프로그램을 운영하는 미술관이다. 예송미술관에서 이번 기간은 4명의 청년 작가의 작품으로 좀 더 깊이있는 시간을 만들 수 있다."
tourapi.loc[tourapi['place_name']=='결성농요농사박물관', 'description'] = "결성농요농사박물관은 홍성군 결성면에 위치한 농업박물관으로, 농사유물과 선사유물 등을 전시하고 있다. 농경생활의 변천사를 한눈에 볼 수 있도록 조성되어 있으며, 농업문화 발굴과 보존을 목표로 하고 있다."
tourapi.loc[tourapi['place_name']=='김포독립운동기념관', 'description'] = "2013년 3월 1일 양촌읍에 개관한 김포시 독립운동기념관은 건축 연면적 2058㎡(623평)에 지상 2층 규모로 건립되었으며 이 지역에서 일어난 독립운동을 재현한 영상과 독립운동을 주도한 인물들을 소개하는 전시관, 일제의 고문실태, 만세운동 체험관 등이 들어서 있다."
tourapi.loc[tourapi['place_name']=='여수자동차극장', 'description'] = "대형 스크린으로 프라이빗한 영화 관람을 즐길 수 있는 자동차 극장이다."

In [45]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_wikipedia_summary

tourist_spot = "잠월미술관"
summary = get_wikipedia_summary(tourist_spot)


In [46]:
tourapi[tourapi['parking'].isnull()]

,place_name,address,latitude,longitude,available_time,parking,description,zip_code,manager,phone,...,day_off,fee,discount_info,viewing_time,parking_fee,stroller,pet,credit_card,details,available_time_v1
4,김포국제조각공원,경기도 김포시 월곶면 용강로13번길 38,37.720951,126.555680,NaN,NaN,김포조각공원은 통일을 테마로 만들어진 세계 유일의 테마공원이다. 민족분단의 현장인 ...,10024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실:있음\n입 장 료:무료\n이용가능시설:국제조각, 산책로, 청소년수련원, 눈썰...","```json\n{\n ""월"": ""정보 없음"",\n ""화"": ""정보 없음..."
5,김포국제조각공원,경기도 김포시 월곶면 용강로13번길 38,37.720951,126.555680,평일 00:00~24:00,NaN,김포조각공원은 통일을 테마로 만들어진 세계 유일의 테마공원이다. 민족분단의 현장인 ...,10024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,없음,NaN,없음,"화장실:있음\n입 장 료:무료\n이용가능시설:국제조각, 산책로, 청소년수련원, 눈썰...","```json\n{\n ""월"": ""00:00~24:00"",\n ""화"": ..."
39,익산국악원,전북특별자치도 익산시 평동로9길 66-3 (갈산동),35.937516,126.951498,09:00~18:00 <br>\n※ 교육 일정에 따라 변동 가능,NaN,"익산은 우도농악의 발상지가 있는 곳으로 판소리계에서도 권삼득, 신만엽, 정춘풍, 유...",54601,NaN,NaN,...,※ 교육 일정에 따라 상이,NaN,※ 교육과정에 따라 상이,NaN,NaN,없음,NaN,없음,NaN,"```json\n{\n ""월"": ""09:00~18:00"",\n ""화"": ..."
40,성산아트홀,경상남도 창원시 의창구 중앙대로 181,35.230094,128.683650,NaN,NaN,성산아트홀은 21세기 창원시의 외형적 성장만이 아닌 일상속에서 쉽게 어울릴 수 있는...,51435,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,없음,NaN,없음,사업내용:공연 및 전시\n,"```json\n{\n ""월"": ""정보 없음"",\n ""화"": ""정보 없음..."
107,한국만화박물관,경기도 부천시 길주로 1(상동),37.508151,126.742450,NaN,NaN,"이곳은 국내 최초, 최대의 만화 전문 박물관으로 각종 희귀 자료와 원로작가들의 원화...",14505,한국만화영상진흥원,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실:있음\n입 장 료:[일반권]<br>\n5,000원<br>\n[가족권]<br>...","```json\n{\n ""월"": ""정보 없음"",\n ""화"": ""정보 없음..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,김동명문학관,강원특별자치도 강릉시 사천면 샛돌1길 30-2,37.820604,128.837695,10:00~18:00,NaN,시인 김동명의 문학관은 강원도 강릉시 사천면 노동리에 있다. 김동명은 전원적인 서정...,25437,NaN,NaN,...,"매주 월요일, 화요일",NaN,NaN,NaN,NaN,없음,없음,없음,NaN,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""휴무"",\n ..."
2684,영월음향역사박물관,강원특별자치도 영월군 김삿갓면 내리계곡로 18,37.070208,128.755500,10:00 ~ 18:00,NaN,영월음향역사박물관은 폐교를 활용한 130평방미터 공간에 1920년대부터 사랑을 받아...,26247,NaN,NaN,...,"매주 월요일, 1월1일, 설/추석 연휴","일반 5,000원",NaN,NaN,NaN,없음,없음,없음,NaN,"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:00 ~ ..."
2686,동광극장,경기도 동두천시 생연동828-2,37.908855,127.052016,NaN,NaN,동광극장은 옛날 모습을 그대로 간직한 얼마 남지 않은 단관극장이다. 과거에 사용하던...,11331,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,가능,NaN,"```json\n{\n ""월"": ""정보 없음"",\n ""화"": ""정보 없음..."
2689,LG상남도서관,서울특별시 종로구 창덕궁1길 18 LG상남도서관,37.579692,126.988467,NaN,NaN,LG 상남도서관은 국내 최초의 ‘디지털 도서관’이자 세계 최초의 ‘책 읽어주는 도서...,3057,NaN,NaN,...,NaN,무료,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"```json\n{\n ""월"": ""정보 없음"",\n ""화"": ""정보 없음..."


In [ ]:
import pandas as pd

parking = pd.read_excel("data/dataset/etc/전국주차장정보표준데이터-20241210.xls")
parking.columns = parking.iloc[0]
parking = parking[1:].reset_index(drop=True)
parking['위도'] = pd.to_numeric(parking['위도'], errors="coerce")
parking['경도'] = pd.to_numeric(parking['경도'], errors="coerce")
parking = parking.dropna(subset=['위도', '경도'])

In [48]:
parking.head(2)

,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,...,결제방법,특기사항,관리기관명,전화번호,위도,경도,장애인전용주차구역보유여부,데이터기준일자,제공기관코드,제공기관명
0,204-2-000023,아래숯골,공영,노외,경기도 성남시 수정구 성남대로 1259,NaN,150,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.442288,127.126662,Y,2024-05-31,3780000,경기도 성남시
1,204-2-000024,태평4동,공영,노외,경기도 성남시 수정구 태평로 109,NaN,89,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.451234,127.138580,Y,2024-05-31,3780000,경기도 성남시


In [49]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import find_nearest_parking

nearest_parking = find_nearest_parking(poi_lat=37.610198, poi_lon=127.040989, parking_data=parking)

In [50]:
nearest_parking

'동방고개 공영 주차장'

In [51]:
import pandas as pd
import numpy as np

def combine_columns(row):
    additional_info = []

    # fee 추가
    if pd.notna(row['fee']):
        additional_info.append(f"요금: {row['fee']}")
    
    # discount_info 추가
    if pd.notna(row['discount_info']):
        additional_info.append(f"할인정보: {row['discount_info']}")
    
    # stroller 추가
    if pd.notna(row['stroller']):
        additional_info.append(f"유모차: {row['stroller']}")
    
    # pet 추가
    if pd.notna(row['pet']):
        additional_info.append(f"반려동물: {row['pet']}")
    
    details = row['details'] if pd.notna(row['details']) else ""
    
    combined_details = details + ('\n' if details and additional_info else '') + '\n'.join(additional_info)
    return combined_details

tourapi['details'] = tourapi.apply(combine_columns, axis=1)

tourapi = tourapi.drop(columns=['fee', 'discount_info', 'stroller', 'pet'])


In [52]:
tourapi['source'] = 'tourAPI'
tourapi['category'] = 'cultural_facilities'

In [53]:
tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'available_time_v1', 'parking', 'description', 'details']]

In [56]:
tourapi.head(2)

,source,category,place_name,address,latitude,longitude,available_time,available_time_v1,parking,description,details
0,tourAPI,cultural_facilities,금나래아트홀,서울특별시 금천구 시흥대로73길 70 금천구청종합청사,37.456493,126.895521,공연마다 상이함,"```json\n{\n ""월"": ""영업시간"",\n ""화"": ""영업시간"",...",가능 (금천구청 지하주차장),"금나래아트홀은 클래식, 연극, 뮤지컬 등 다양한 장르의 공연을 소화할 수 있는 다목...","대관안내:공연장, 갤러리 대관 가능<br />\n※ 대관문의 : 금나래 아트홀 운영..."
1,tourAPI,cultural_facilities,상상톡톡 미술관,서울특별시 강북구 월계로 173(번동),37.624210,127.038297,10:00~18:00 (입장마감은 17:30),"```json\n{\n ""월"": ""휴무"",\n ""화"": ""10:00~18...",가능 (북서울꿈의숲 서문 주차장),상상 톡톡 미술관은 강북구 번동 꿈의 숲 아트센터 내에 있는 미술관을 새로 단장하여...,"체험프로그램:전시에 따라 체험프로그램 변동\n\n요금: [입장료]<br>-12,00..."
